In [435]:
import pandas as pd
import os
import openpyxl

In [436]:
# file_path = 'data/mini/NL RIVM-NEVO_mini.txt'
file_path = 'data/NL RIVM-NEVO 2008-05-22.txt'


with open(file_path, 'r', encoding='latin-1') as file:
    lines = file.readlines()

headers = lines[0].strip().split('\t')

data = []
for line in lines[1:]:
    fields = line.strip().split('\t')
    if len(fields) < len(headers):
        fields += [''] * (len(headers) - len(fields))
    data.append(fields)

# print(data)

df_nevo_langal = pd.DataFrame(data, columns=headers)

def convert_to_set(langual_codes):
    return set(langual_codes.split())

# Apply the function to the LANGUALCODES column
df_nevo_langal['LANGUALCODES'] = df_nevo_langal['LANGUALCODES'].apply(convert_to_set)

df_nevo_langal.head()

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS
0,945,Aardappel driekantjes bereid in diamantvet,Potatoes triangulardeep fat fried,"{B1218, A0152, A0706, A0829}",
1,947,Aardappel driekantjes bereid in oven,Potatoes triangular prepared in oven,"{B1218, A0152, A0706, A0829}",
2,946,Aardappel driekantjes bereid in zonnebloemolie,Potatoes triangular prep in sunflower oi,"{B1218, A0152, A0706, A0829}",
3,1457,Aardappelen gebakken,Potatoes fried,"{P0078, F0014, K0003, A0829, G0026, P0024, A01...",
4,982,Aardappelen gekookt,Potatoes boiled,"{P0078, F0014, G0015, K0003, A0829, P0024, A01...",


In [445]:
# Path to your Excel file
excel_path = 'data/Frida_5.1_November_2023.xlsx'

# Create a Pandas Excel file object
xls = pd.ExcelFile(excel_path)

# Load each sheet into a separate DataFrame
for sheet_name in xls.sheet_names:
    if sheet_name == "Food":
        df_frida_langal = pd.read_excel(xls, sheet_name)

# df_frida_langal

def convert_to_set(langual_codes):
    if isinstance(langual_codes, str):
        return set(langual_codes.split(','))
    else:
        return set()  # or np.nan if you prefer to keep NaN values

# # Apply the function to the LANGUALCODES column
df_frida_langal['LangualCode'] = df_frida_langal['LangualCode'].apply(convert_to_set)

# df_frida_langal

In [438]:
# # file_path = 'data/mini/DK DTU_FDB 2008-06-26.txt'
# file_path = 'data/DK DTU_FDB 2008-06-26.txt'

# with open(file_path, 'r', encoding='latin-1') as file:
#     lines = file.readlines()

# headers = lines[0].strip().split('\t')

# data = []
# for line in lines[1:]:
#     fields = line.strip().split('\t')
#     if len(fields) < len(headers):
#         fields += [''] * (len(headers) - len(fields))
#     data.append(fields)

# df_frida_langal = pd.DataFrame(data, columns=headers)


# def convert_to_set(langual_codes):
#     return set(langual_codes.split())

# # Apply the function to the LANGUALCODES column
# df_frida_langal['LANGUALCODES'] = df_frida_langal['LANGUALCODES'].apply(convert_to_set)

# df_frida_langal

In [439]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def langual_similarity(code1, code2):
    set1 = set(code1.split())
    set2 = set(code2.split())
    return jaccard_similarity(set1, set2)

# EXAMPLE: calculating similarity
# similarity = jaccard_similarity(
#     df_nevo_langal['LANGUALCODES'][0],
#     df_nevo_langal['LANGUALCODES'][1]
# )


In [448]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    # REMOVED; INACCURATE
    # return len(intersection) / len(union) 
    return len(intersection) / len(set1)
    

def langual_similarity(code1, code2):
    set1 = set(code1.split())
    set2 = set(code2.split())
    return jaccard_similarity(set1, set2)

# EXAMPLE: calculating similarity
# similarity = jaccard_similarity(
#     df_nevo_langal['LANGUALCODES'][0],
#     df_nevo_langal['LANGUALCODES'][1]
# )

# Calculate Jaccard similarity between the first row and the other rows
first_row_set = df_frida_langal['LangualCode'][0]
df_nevo_langal['SIMILARITYSCORE'] = df_nevo_langal['LANGUALCODES'].apply(lambda x: jaccard_similarity(first_row_set, x))
print(df_frida_langal['FoodName'][0])
print(df_frida_langal['LangualCode'][0])
# Set pandas options to display all rows
pd.set_option('display.max_rows', None)

# Sort the DataFrame based on SIMILARITY_TO_FIRST column from greatest to least
df_nevo_langal = df_nevo_langal.sort_values(by='SIMILARITYSCORE', ascending=False)

df_nevo_langal.head(50)

Strawberry, raw
{'A0833', 'A0671', 'E0150', 'A0455', 'N0001', 'M0001', 'B1393', 'F0003', 'A0707', 'K0003', 'C0167', 'H0003', 'G0003', 'A0732', 'A0143', 'P0024', 'J0003', 'Z0001', 'A0629'}


,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS,SIMILARITYSCORE
14,148,Aardbeien,Strawberries,"{F0003, A0833, A0707, G0003, K0003, A0143, P00...",,0.631579
602,690,Guave,Guava,"{F0003, C0139, A0833, A0707, K0003, A0143, P00...",,0.578947
114,151,Banaan,Banana,"{F0003, C0138, A0833, A0707, K0003, B1266, A01...",,0.578947
218,157,Bramen,Blackberries,"{F0003, A0833, A0707, G0003, K0003, A0143, P00...",,0.578947
1049,165,Mandarijn,Tangerines,"{F0003, C0229, A0833, B1429, A0707, K0003, A01...",,0.578947
391,160,Druiven witte/blauwe m schil,Grapes white/black with skin,"{F0003, A0833, A0707, G0003, K0003, A0143, P00...",,0.578947
1932,694,Tamarinde,Tamarind,"{F0003, C0229, B1695, A0833, A0707, K0003, A01...",,0.578947
1431,169,Perzik z schil,Peach without skin,"{F0003, C0229, B1350, A0833, A0707, K0003, A01...",,0.578947
1413,168,Peer z schil,Pear without skin,"{F0003, C0229, A0833, A0707, K0003, A0143, P00...",,0.578947
28,150,Ananas,Pineapple,"{F0003, C0229, A0833, A0707, K0003, A0143, P00...",,0.578947


In [ ]:
# Calculate Jaccard similarity between the first row and the other rows
first_row_set = df_nevo_langal['LANGUALCODES'][0]
df_frida_langal['SIMILARITYSCORE'] = df_frida_langal['LANGUALCODE'].apply(lambda x: jaccard_similarity(first_row_set, x))

# Sort the DataFrame based on SIMILARITY_TO_FIRST column from greatest to least
df_frida_langal = df_frida_langal.sort_values(by='SIMILARITYSCORE', ascending=False)

df_frida_langal.head(50)

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS,SIMILARITYSCORE
818,0819,"Kartoffel, ny (maj til september), rå","Potato, new (May to September), raw","{A0650, F0003, G0003, A0731, A0629, A0705, A08...",,0.75
114,0115,"Kartoffel, uspec., rå","Potato, raw","{A0650, F0003, G0003, A0731, A0629, A0705, A08...",,0.75
1208,1210,"Knorr Kartoffelmos komplet, pulver - dk art .","Knorr Potato mash, powder","{A0731, A0706, A0455, H0138, N0001, M0001, F00...",,0.75
1261,1263,"Kartoffel, bagt","Potato, baked","{A0650, G0005, F0014, K0003, A0731, A0629, A07...",,0.75
1260,1262,"Kartoffel, kogt","Potato, boiled","{A0650, K0003, A0731, A0629, A0705, A0829, C02...",,0.75
658,0659,"Batat, sød kartoffel, rå","Sweetpotato, raw","{F0003, B1106, A0681, K0003, A0731, A0629, A08...",,0.75
1222,1224,"Knorr Kartoffelmos komplet, pulver - DK Art. 3...","Knorr Potato mash, powder","{A0731, A0706, A0455, H0138, N0001, M0001, F00...",,0.75
820,0821,"Kartoffel, gammel (februar - juni), rå","Potato, old (February to June), raw","{A0650, F0003, G0003, A0731, A0629, A0705, A08...",,0.75
819,0820,"Kartoffel, efterår (oktober til januar), rå","Potato, automn (October to January), raw","{A0650, F0003, G0003, A0731, A0629, A0705, A08...",,0.75
100,0101,"Jordskok, rå","Jerusalem artichoke, raw","{A0650, F0003, K0003, A0731, A0629, A0705, A08...",,0.50
